<a href="https://colab.research.google.com/github/anshupandey/Natural_language_Processing/blob/master/TextGeneration_with_LSTM_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text Generation with LSTM using Pytorch

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from collections import Counter
from itertools import chain

In [15]:
# Sample Data
data = "The quick brown fox jumps over the lazy dog."

# Tokenization and Vocabulary Creation
words = data.split()
vocab = list(set(words))
word2idx = {word: idx for idx, word in enumerate(vocab)}
idx2word = {idx: word for word, idx in word2idx.items()}
vocab_size = len(vocab)
sequences = [[word2idx[word] for word in words[:i + 1]] for i in range(1, len(words))]


In [16]:
# Dataset Preparation
class TextDataset(Dataset):
    def __init__(self, sequences):
        self.sequences = sequences

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        sequence = self.sequences[idx]
        return torch.tensor(sequence[:-1], dtype=torch.long), torch.tensor(sequence[-1], dtype=torch.long)


In [17]:
# Model Definition
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        embedded = self.embedding(x)
        output, (hidden, cell) = self.lstm(embedded)
        output = self.fc(output[:, -1, :])
        return output

In [18]:
# Parameters
embedding_dim = 50
hidden_dim = 100
data_loader = DataLoader(TextDataset(sequences), batch_size=1, shuffle=True)

# Training
model = LSTMModel(vocab_size, embedding_dim, hidden_dim)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

for epoch in range(50):
    for X, y in data_loader:
        optimizer.zero_grad()
        output = model(X)
        loss = criterion(output, y)
        loss.backward()
        optimizer.step()

In [19]:
# Prediction
input_text = "The quick brown"
input_seq = [word2idx[word] for word in input_text.split()]
input_seq = torch.tensor(input_seq).unsqueeze(0)
output = model(input_seq)
predicted_word = idx2word[torch.argmax(output, dim=1).item()]
print(predicted_word)

fox


In [9]:
!pip install torchviz

In [20]:

from torchviz import make_dot
# Display Model Architecture using torchviz
X, y = next(iter(data_loader))
output = model(X)
make_dot(output, params=dict(model.named_parameters())).render("model_arch", format="png")

'model_arch.png'